### checkking for ollama

In [2]:
from langchain_community.llms import Ollama

llm = Ollama(model="gemma2:2b")

response = llm.invoke("Why won't i find someone?")

In [3]:
print(response)

It's tough to feel like you're not finding love or connection.  While I can't give you specific answers about your dating life, there are many possible reasons why this might be happening, and some things to consider: 

**Understanding Yourself & Your Goals:**

* **Clear Vision:**  Do you have a clear idea of what kind of partner you want and what you're looking for in a relationship? Defining these qualities will help you focus your efforts.
* **Self-Discovery:** Are you happy and fulfilled with yourself right now? If not, focusing on personal growth can improve your confidence and attractiveness to others. 
* **Emotional Maturity:**  Are you emotionally available for something deeper? Can you handle challenges and be open to vulnerability?

**About Your Approach:**

* **Networking & Exploration:**  Have you been putting yourself out there in ways that are comfortable and engaging? Joining social groups, attending events, and trying new hobbies can increase your chances of meeting som

### checking for whisper

In [1]:
import whisper

model = whisper.load_model("base")

# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("oh_hello_deadpool.mp3")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

# decode the audio
options = whisper.DecodingOptions()
result = whisper.decode(model, mel, options)

# print the recognized text
print(result.text)

d:\github\Assistant\.venv\Lib\site-packages\whisper\__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Detected language: en
Oh! Oh hello!


### main code

In [2]:
!pip install whisper ollama google-auth-oauthlib google-auth-httplib2 google-api-python-client

### basic working test

In [4]:
import whisper
import ollama
import json
import os

def transcribe_audio(audio_file):
    model = whisper.load_model("base")
    
    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(audio_file)
    audio = whisper.pad_or_trim(audio)
    
    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)
    
    # detect the spoken language
    _, probs = model.detect_language(mel)
    detected_language = max(probs, key=probs.get)
    print(f"Detected language: {detected_language}")
    
    # decode the audio
    options = whisper.DecodingOptions()
    result = whisper.decode(model, mel, options)
    
    return result.text, detected_language

def process_with_ollama(text, language):
    # You can customize this prompt based on your needs
    prompt = f"""
    Language: {language}
    User input: {text}
    
    Analyze the user's input and determine the appropriate action or response. 
    If it's a command or question, provide a suitable answer or action plan.
    If it's a general statement, engage in a contextually appropriate conversation.
    """
    
    response = ollama.chat(model='gemma2:2b', messages=[
        {
            'role': 'system',
            'content': 'You are a helpful voice assistant.'
        },
        {
            'role': 'user',
            'content': prompt
        }
    ])
    
    return response['message']['content']

def text_to_speech(text):
    # Placeholder for text-to-speech functionality
    # You can integrate a TTS library or service here
    print("Text-to-speech output:", text)

def main():
    audio_file = "oh_hello_deadpool.mp3"
    
    # Step 1: Transcribe audio
    transcribed_text, detected_language = transcribe_audio(audio_file)
    print("Transcribed text:", transcribed_text)
    
    # Step 2: Process with Ollama
    ollama_response = process_with_ollama(transcribed_text, detected_language)
    print("Ollama response:", ollama_response)
    
    # Step 3: Convert response to speech
    text_to_speech(ollama_response)

if __name__ == "__main__":
    main()

Detected language: en
Transcribed text: Oh! Oh hello!
Ollama response: The user's input "Oh! Oh hello!" is an enthusiastic greeting.  

**Action:** I should respond with a friendly greeting back. 

Here are some options:

* **Option 1 (Casual):**  "Hey there!" 
* **Option 2 (More formal):** "Hello! How can I help you today?"
* **Option 3 (Humorous):** "Well, hello! Did a squirrel get into your voicebox?"


Which response would you prefer? 😊 

Text-to-speech output: The user's input "Oh! Oh hello!" is an enthusiastic greeting.  

**Action:** I should respond with a friendly greeting back. 

Here are some options:

* **Option 1 (Casual):**  "Hey there!" 
* **Option 2 (More formal):** "Hello! How can I help you today?"
* **Option 3 (Humorous):** "Well, hello! Did a squirrel get into your voicebox?"


Which response would you prefer? 😊 



### test 2

In [5]:
!pip install whisper ollama google-auth-oauthlib google-auth-httplib2 google-api-python-client

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached protobuf-5.27.3-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached cachetools-5.4.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
   ---------------------------------------- 0.0/12.1 MB ? eta -:--:--
   ------- -------------------------------- 2.4/12.1 MB 12.2 MB/s eta 0:00:01
   --------------- ------------------------ 4.7/12.1 MB 11.9 MB/s eta 0:00:01
   ------------------------ --------------- 7.3/12.1 MB 11.6 MB/s eta 0:00:01
   -------------------------------- ------- 9.7/12.1 MB 11.6 MB/s eta 0:00:01
   ---------------------------------------  12.1/12.1 MB 11.6 MB/s eta 0:00:01

In [7]:
!pip install pytz

  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)


In [15]:
import whisper
import ollama
from datetime import datetime
import pytz
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow

SCOPES = ['https://www.googleapis.com/auth/calendar']

def transcribe_audio(audio_file):
    if not os.path.exists(audio_file):
        raise FileNotFoundError(f"The audio file '{audio_file}' does not exist.")
    
    try:
        model = whisper.load_model("base")
        audio = whisper.load_audio(audio_file)
        audio = whisper.pad_or_trim(audio)
        mel = whisper.log_mel_spectrogram(audio).to(model.device)
        options = whisper.DecodingOptions()
        result = whisper.decode(model, mel, options)
        return result.text
    except Exception as e:
        raise RuntimeError(f"Failed to transcribe audio: {str(e)}")


def natural_language_understanding(text):
    prompt = f"""
    Analyze the following user input and extract information for creating a calendar event:
    User input: {text}

    Please provide the following details in JSON format:
    1. Event title
    2. Start date and time
    3. End date and time (if specified, otherwise assume 1 hour duration)
    4. Location (if specified)
    5. Description (if any additional details are provided)

    If the input doesn't contain enough information for a calendar event, respond with {{"error": "Not enough information"}}.
    """
    
    response = ollama.chat(model='gemma2:2b', messages=[
        {
            'role': 'system',
            'content': 'You are a helpful assistant that extracts calendar event information from user input.'
        },
        {
            'role': 'user',
            'content': prompt
        }
    ])
    
    return response['message']['content']

def create_calendar_event(event_details):
    print("started creating calendar event")
    creds = None
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('client_secret.json', SCOPES)
        creds = flow.run_local_server(port=0)
    
    service = build('calendar', 'v3', credentials=creds)
    
    event = {
        'summary': event_details['title'],
        'location': event_details.get('location', ''),
        'description': event_details.get('description', ''),
        'start': {
            'dateTime': event_details['start'],
            'timeZone': 'Europe/Oslos',
        },
        'end': {
            'dateTime': event_details['end'],
            'timeZone': 'Europe/Oslos',
        },
    }
    print("almost created calendar event")

    event = service.events().insert(calendarId='primary', body=event).execute()
    return f"Event created: {event.get('htmlLink')}"

def text_to_speech(text):
    # Placeholder for text-to-speech functionality
    print("Assistant:", text)

def main():
    audio_file = "recordings\meet_at_8.mp3"
    
    try:
        # Step 1: Speech-to-Text
        transcribed_text = transcribe_audio(audio_file)
        print("Transcribed text:", transcribed_text)
        
        # Step 2: Natural Language Understanding
        event_info_str = natural_language_understanding(transcribed_text)
        
        # Step 3: Intent Classification and Action
        try:
            event_info = json.loads(event_info_str)
            if "error" in event_info:
                response = "I'm sorry, but I couldn't extract enough information to create a calendar event. Could you please provide more details?"
            else:
                result = create_calendar_event(event_info)
                response = f"I've created the event for you. {result}"
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON: {str(e)}")
            print(f"Raw response from LLM: {event_info_str}")
            response = "I'm sorry, but I encountered an error while processing your request. Could you please try rephrasing your input?"
        
        # Step 4: Text-to-Speech
        text_to_speech(response)
    
    except FileNotFoundError as e:
        print(f"Error: {str(e)}")
        print("Please make sure the audio file exists and the path is correct.")
    except RuntimeError as e:
        print(f"Error: {str(e)}")
        print("There was an issue processing the audio. Please try again or use a different audio file.")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
        print("Please try again or contact support if the issue persists.")

if __name__ == "__main__":
    main()

Transcribed text: Hi, I will meet Rahul today around 8. So yeah, thanks.
Error parsing JSON: Expecting value: line 1 column 1 (char 0)
Raw response from LLM: ```json
{"event": {"title": "Meeting with Rahul", "start_date_time": "2023-10-27T08:00:00", "end_date_time": "2023-10-27T09:00:00", "location": null, "description": "Meeting with Rahul at around 8"}}
``` 


**Explanation:**

* **Event title:**  "Meeting with Rahul" is extracted from the user input.
* **Start date and time:**  "2023-10-27T08:00:00" assumes this refers to the exact meeting start time, based on "today". 
* **End date and time:** "2023-10-27T09:00:00"  assumes a one-hour duration after the start. 
* **Location:** No specific location is mentioned in the user input, so we set the "location" field to null for now. You can update this if you receive more details from the user about where the meeting will take place. 
* **Description:**  The sentence "So yeah, thanks" provides no additional information about the meeting b

In [16]:
import whisper
import ollama
from datetime import datetime
import pytz
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow

def create_calendar_event(event_details):
    print("started creating calendar event")
    creds = None
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('client_secret.json', SCOPES)
        creds = flow.run_local_server(port=0)
    
    service = build('calendar', 'v3', credentials=creds)
    
    event = {
        'summary': event_details['title'],
        'location': event_details.get('location', ''),
        'description': event_details.get('description', ''),
        'start': {
            'dateTime': event_details['start'],
            'timeZone': 'Europe/Oslos',
        },
        'end': {
            'dateTime': event_details['end'],
            'timeZone': 'Europe/Oslos',
        },
    }
    print("almost created calendar event")

    event = service.events().insert(calendarId='primary', body=event).execute()
    return f"Event created: {event.get('htmlLink')}"


def main():
    json = {"event": {"title": "Meeting with Rahul", "start_date_time": "2023-10-27T08:00:00", "end_date_time": "2023-10-27T09:00:00", "location": null, "description": "Meeting with Rahul at around 8"}}
    event_info = json.loads(json)
    result = create_calendar_event(event_info)
    response = f"I've created the event for you. {result}"
    print(response)


SyntaxError: incomplete input (2179933577.py, line 40)